## Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login
```

or

```bash
az login --use-device-code
```

# Azure AI Agent with Bing Grounding Example

This notebook demonstrates how to create an Azure AI agent that uses Bing Grounding search to find real-time information from the web.

## Features Covered:
- Setting up Bing Grounding search integration
- Using HostedWebSearchTool for web searches
- Creating agents that can access current web information
- Proper resource management with async context managers

## Prerequisites

Before running this notebook, ensure you have:
- Azure CLI installed and authenticated (`az login --use-device-code`)
- Access to an Azure AI Foundry project with deployed models
- Environment variables set up (see Initial Setup section below)

This example demonstrates Azure AI agents with Bing Grounding for web search capabilities. The agent can search the web for current information and provide informed responses.

## Import Libraries

Import the required libraries for Azure AI agent functionality.

In [1]:
import os
from pathlib import Path
from agent_framework import ChatAgent, HostedWebSearchTool
from agent_framework_azure_ai import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

## Initial Setup

Initialize the Azure AI project client and set up authentication.

In [2]:
from pathlib import Path  # For working with file paths
import os  # For environment variables
from dotenv import load_dotenv  # For loading environment variables from .env file

# Get the path to the .env file which is in the parent directory
notebook_path = Path().absolute()  # Get absolute path of current notebook
parent_dir = notebook_path.parent  # Get parent directory
load_dotenv('../../.env')  # Load environment variables from .env file

# Get configuration from environment variables
project_endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")
model_deployment = os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o")

# Validate and clean endpoint (remove any inline comments)
if project_endpoint and '#' in project_endpoint:
    print("⚠️  WARNING: Endpoint contains a '#' character - cleaning it...")
    project_endpoint = project_endpoint.split('#')[0].strip()

print(f"🔗 Project Endpoint: {project_endpoint}")
print(f"🤖 Model Deployment: {model_deployment}")

🔗 Project Endpoint: https://demopocaifoundry.services.ai.azure.com/api/projects/demoproject
🤖 Model Deployment: gpt-4o


## Retrieve Bing Connection ID

We'll use the Azure AI Project Client to get the Bing connection and retrieve its full resource ID:

In [4]:
from azure.ai.projects import AIProjectClient
from azure.identity import AzureCliCredential

# Get the Bing connection name from environment
bing_connection_name = os.environ.get("BING_CONNECTION_NAME")

if not bing_connection_name:
    raise ValueError("BING_CONNECTION_NAME environment variable is not set")

print(f"🔍 Looking for connection: {bing_connection_name}")

# Create project client to retrieve the connection
project_client = AIProjectClient(
    credential=AzureCliCredential(),
    subscription_id=os.environ.get("AZURE_SUBSCRIPTION_ID"),
    resource_group_name="demoaifoundry",
    project_name="demoproject",
    endpoint=project_endpoint,  # Add the endpoint parameter
)

# Get the connection and extract its ID
bing_connection = project_client.connections.get(name=bing_connection_name)
bing_connection_id = bing_connection.id

print(f"✅ Bing connection retrieved successfully")
print(f"Connection ID: {bing_connection_id}")

🔍 Looking for connection: foundrysearch
✅ Bing connection retrieved successfully
Connection ID: /subscriptions/93c5449c-bbb2-4249-9461-ddf749e03430/resourceGroups/demoaifoundry/providers/Microsoft.CognitiveServices/accounts/demopocaifoundry/projects/demoproject/connections/foundrysearch


## Create Bing Grounding Search Tool

Now create the `HostedWebSearchTool` with the retrieved connection ID:

In [5]:
# Create Bing Grounding search tool with the connection ID
bing_search_tool = HostedWebSearchTool(
    name="Bing Grounding Search",
    description="Search the web for current information using Bing",
    connection_id=bing_connection_id,  # Pass the retrieved connection ID
)

print("✅ Bing search tool created successfully")

✅ Bing search tool created successfully


## Create and Run the Agent

Now let's create an Azure AI agent that can use the Bing search tool to find current information from the web:

In [6]:
async def main() -> None:
    """Main function demonstrating Azure AI agent with Bing Grounding search."""
    
    # Use AzureAIAgentClient as async context manager for automatic cleanup
    async with (
        AzureAIAgentClient(
            async_credential=AzureCliCredential(),
            project_endpoint=project_endpoint,
            model_deployment_name=model_deployment
        ) as client,
        ChatAgent(
            chat_client=client,
            name="BingSearchAgent",
            instructions=(
                "You are a helpful assistant that can search the web for current information. "
                "Use the Bing search tool to find up-to-date information and provide accurate, "
                "well-sourced answers. Always cite your sources when possible."
            ),
            tools=bing_search_tool,
        ) as agent,
    ):
        # Demonstrate agent capabilities with web search
        print("=== Azure AI Agent with Bing Grounding Search ===\n")

        user_input = "What is the most popular programming language?"
        print(f"User: {user_input}")
        response = await agent.run(user_input)
        print(f"Agent: {response.text}\n")

## Execute the Example

Run the main function to see the agent in action:

In [7]:
# Run the main function
await main()

=== Azure AI Agent with Bing Grounding Search ===

User: What is the most popular programming language?
Agent: As of February 2026, Python remains the most popular programming language globally due to its dominance in fields like Artificial Intelligence, Machine Learning, and Data Science. It holds the top spot in metrics like the TIOBE Index with a significant percentage of developer interest. JavaScript follows closely, being the most widely used for web development. Rust is the most admired language for system programming, while TypeScript continues to gain traction as a safer JavaScript extension【3:0†source】【3:1†source】【3:2†source】.



## Try Additional Queries

Let's test the agent with more queries that require current information:

In [8]:
async def test_more_queries():
    """Test the agent with additional queries that require web search."""
    
    async with (
        AzureAIAgentClient(
            async_credential=AzureCliCredential(),
            project_endpoint=project_endpoint,
            model_deployment_name=model_deployment
        ) as client,
        ChatAgent(
            chat_client=client,
            name="BingSearchAgent",
            instructions=(
                "You are a helpful assistant that can search the web for current information. "
                "Use the Bing search tool to find up-to-date information and provide accurate, "
                "well-sourced answers. Always cite your sources when possible."
            ),
            tools=bing_search_tool,
        ) as agent,
    ):
        queries = [
            "What are the latest trends in artificial intelligence?",
            "What is the current weather in Seattle?",
            "What are the latest Microsoft announcements?"
        ]
        
        for query in queries:
            print(f"\n--- Query: {query} ---")
            response = await agent.run(query)
            print(f"Agent: {response.text}")
            print("-" * 50)

# Run additional queries
await test_more_queries()


--- Query: What are the latest trends in artificial intelligence? ---
Agent: The latest trends in artificial intelligence (AI) reflect its rapid evolution into a collaborative and transformative force across various sectors. Key developments for 2026 include:

1. **Agentic AI**: AI systems are becoming more autonomous, capable of making decisions, setting goals, and managing tasks with limited human intervention. This evolution supports industries like customer service, supply chain management, and finance【3:4†source】【3:3†source】.

2. **AI as a Digital Coworker**: AI agents are increasingly viewed as teammates, enhancing productivity and enabling small teams to perform tasks that were previously unmanageable. These agents assist with content generation, data analysis, and task personalization【3:0†source】【3:4†source】.

3. **Generative AI for Organizations**: The focus is shifting from generative AI as an individual tool to its broader organizational applications, offering capabilities 

## Key Takeaways

1. **Bing Grounding Integration**: The `HostedWebSearchTool` provides seamless integration with Bing search capabilities
2. **Connection Retrieval**: Use the Azure AI Project Client to retrieve the full connection ID from the connection name
3. **Environment Configuration**: Set `BING_CONNECTION_NAME` in your `.env` file (just the name, not the full resource ID)
4. **Real-time Information**: Agents can now access current web information, making them more useful for time-sensitive queries
5. **Source Citation**: The agent is instructed to cite sources, promoting transparency and reliability
6. **Resource Management**: Using async context managers ensures proper cleanup of resources
7. **Tool Integration**: The Bing search tool seamlessly integrates with the agent's existing capabilities

## Troubleshooting

If you encounter issues:
1. **Authentication**: Ensure you're logged in with `az login`
2. **Connection Name**: Set `BING_CONNECTION_NAME` in your `.env` file with just the connection name (e.g., `foundrysearch`)
3. **Azure AI Project**: Verify that Bing Grounding connection is properly configured in your Azure AI Foundry project
4. **Permissions**: Ensure you have the necessary permissions to access the connected Bing search resource
5. **Connection Format**: The connection ID is automatically retrieved - you don't need to manually construct the full Azure Resource Manager path